In [1]:
import sys
import os
%load_ext autoreload
%autoreload 2

sys.path.append('../')
os.chdir('../')

In [2]:
import re

string = "res=[0, 1, 2, 5]"
pattern = r"res=\[(.*?)\]"
result = re.findall(pattern, string)[0]
lst = [int(num) for num in result.split(", ")]
print(lst)


[0, 1, 2, 5]


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re

critical_keys1 = ['parity', 'paridy']
critical_keys2 = ['nc']

global_path = './outputs'

for c1 in critical_keys1:
    for c2 in critical_keys2:
        for c2_value in [True, False]:
            all_result_paths = glob.glob(global_path + '/pari*_exp/**/result.csv', recursive=True)
            all_result_paths = [p for p in all_result_paths if c1 in p]
            all_result_paths = [p for p in all_result_paths if (c2 in p)==c2_value]

            result_summary = {}
            n_layers = 6
            for result_path in all_result_paths:
                result = pd.read_csv(result_path)
                top_test_result = result['test_acc'].max()
                pe_type = 'NoPE' if 'nope' in result_path else 'Original'
                pattern = r"res=\[(.*?)\]"
                residual_list = re.findall(pattern, result_path)
                if len(residual_list) == 0:
                    residual_list = np.arange(n_layers)
                else:
                    residual_list = residual_list[0]
                    residual_list = [int(num) for num in residual_list.split(", ")] if residual_list!='' else []
                # residual_list = [int(num) for num in residual_list.split(", ")]
                missing_rc = [i for i in range(n_layers) if i not in residual_list]
                missing_rc_str = '{' + ','.join([str(i) for i in missing_rc])+ '}'


                if pe_type not in result_summary:
                    result_summary[pe_type] = {}
                if missing_rc_str not in result_summary[pe_type]:
                    result_summary[pe_type][missing_rc_str] = []
                result_summary[pe_type][missing_rc_str].append(top_test_result)
                if pe_type == 'Original' and missing_rc_str == '{5}':
                    print(result_path, pe_type, missing_rc_str, top_test_result)

            from ast import literal_eval
            mean_results = {}
            for pe_type in result_summary:
                cur_type_dict = result_summary[pe_type].copy()
                for missing_rc in cur_type_dict:
                    cur_type_dict[missing_rc] = len(cur_type_dict[missing_rc])
                mean_results[pe_type] = cur_type_dict

            df = pd.DataFrame(mean_results)
            missing0 = ['{}']
            missing1 = [str({i}) for i in range(n_layers)]
            missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
            missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
            missing4 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3)])+ '}'for i in range(n_layers-3)]
            missing5 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4)])+ '}'for i in range(n_layers-4)]
            from IPython.display import display
            display(df.loc[missing0,:].T)
            display(df.loc[missing1,:].T)
            display(df.loc[missing2,:].T)
            display(df.loc[missing3,:].T)
            display(df.loc[missing4,:].T)
            display(df.loc[missing5,:].T)
            num_seeds = df.values.min()


            from collections import OrderedDict
            final_results = OrderedDict()
            for pe_type in {'Original', 'NoPE'}:
                cur_type_dict = result_summary[pe_type].copy()
                final_dict = {
                    'min': cur_type_dict.copy(),
                    'max': cur_type_dict.copy(),
                    'avg.': cur_type_dict.copy(),
                }
                for missing_rc in cur_type_dict:
                    final_dict['avg.'][missing_rc] = np.mean(cur_type_dict[missing_rc])
                    final_dict['min'][missing_rc] = np.min(cur_type_dict[missing_rc])
                    final_dict['max'][missing_rc] = np.max(cur_type_dict[missing_rc])

                for k in final_dict:
                    final_results[pe_type+' ('+k+')'] = final_dict[k]

            df = pd.DataFrame(final_results)
            missing1 = [str({i}) for i in range(n_layers)]
            missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
            missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
            missing4 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3)])+ '}'for i in range(n_layers-3)]
            missing5 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4)])+ '}'for i in range(n_layers-4)]
            from IPython.display import display
            r1 = df.loc[missing1,:].T
            r2 = df.loc[missing2,:].T
            r3 = df.loc[missing3,:].T
            r4 = df.loc[missing4,:].T
            r5 = df.loc[missing5,:].T
            r0 = df.loc[['{}'],:].T
            display(r0)
            display(r1)
            display(r2)
            display(r3)
            display(r4)
            display(r5)

            # write all 3 to the same csv file
            all_r = pd.concat([r0, r1, r2, r3, r4, r5], axis=1)
            c2 = '_' + c2 if c2_value else ''
            all_r.to_csv(f'{c1}{c2}_{num_seeds}seeds.csv')



./outputs/parity_nc_residual_exp/parity_nc_sd240_T2403281738_res=[0, 1, 2, 3, 4]_ncTrue/parity_nc_sd240_T2403281738_res=[0, 1, 2, 3, 4]_ncTrue/result.csv Original {5} 99.5
./outputs/parity_nc_residual_exp/parity_nc_sd241_T2403290101_res=[0, 1, 2, 3, 4]_ncTrue/parity_nc_sd241_T2403290101_res=[0, 1, 2, 3, 4]_ncTrue/result.csv Original {5} 99.65


,{}
Original,2
NoPE,2


,{0},{1},{2},{3},{4},{5}
Original,2,2,2,2,2,2
NoPE,2,2,2,2,2,2


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
Original,2,2,2,2,2
NoPE,2,2,2,2,2


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
Original,2,2,2,2
NoPE,2,2,2,2


...

,{0},{1},{2},{3},{4},{5}
Original (min),95.700000,93.350000,98.400000,99.150000,99.050000,97.950000
Original (max),98.400000,98.950000,99.050000,99.500000,99.400000,99.200000
Original (avg.),96.733333,97.016667,98.783333,99.350000,99.233333,98.766667
NoPE (min),90.950000,75.550000,73.850000,97.850000,99.000000,94.650000
NoPE (max),98.800000,91.450000,93.450000,99.200000,99.150000,99.300000
NoPE (avg.),94.483333,84.400000,83.283333,98.733333,99.100000,96.866667


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
Original (min),78.40,98.600000,99.050000,99.10,99.300000
Original (max),99.10,99.150000,99.350000,99.50,99.750000
Original (avg.),91.30,98.916667,99.166667,99.35,99.500000
NoPE (min),67.85,72.600000,75.000000,97.10,97.750000
NoPE (max),77.50,99.100000,88.650000,99.35,99.200000
NoPE (avg.),72.45,87.583333,80.400000,98.35,98.683333


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
Original (min),95.05,98.550000,99.25,99.100000
Original (max),99.15,99.500000,99.40,99.600000
Original (avg.),98.30,99.033333,99.30,99.316667
NoPE (min),51.95,98.750000,87.80,84.650000
NoPE (max),94.10,99.250000,99.35,95.050000
NoPE (avg.),80.91,98.983333,92.85,90.583333


,"{0,1,2,3}","{1,2,3,4}","{2,3,4,5}"
Original (min),98.650000,67.100000,51.35
Original (max),99.200000,98.700000,81.45
Original (avg.),98.966667,88.116667,61.70
NoPE (min),54.750000,75.850000,51.35
NoPE (max),85.800000,98.900000,52.85
NoPE (avg.),70.933333,87.016667,51.85


,"{0,1,2,3,4}","{1,2,3,4,5}"
Original (min),51.00,51.350000
Original (max),86.15,52.300000
Original (avg.),73.35,51.666667
NoPE (min),51.80,50.100000
NoPE (max),59.35,52.300000
NoPE (avg.),55.85,51.250000
